#  Memory Optimization for Medium Sized Dataset Using Batch Processing

 I will be optimizing the Lending Club's website data from the timeframe of 2007-2011 with the assumption that I only have 10mb to process and analyze the data.


In [22]:
import pandas as pd
import pprint as pp
pd.options.display.max_columns = 30

Pandas library provides a very easy way to batch process the dataset using chunks. Below split the dataset into 1000 row chunks and determine the total memory footprint as well as the total number of rows.

In [23]:
chunk_iter=pd.read_csv('loans_2007.csv',chunksize=1000)
total_mem=0
total_rows=0
for chunk in chunk_iter:
    total_mem+=chunk.memory_usage(deep=True).sum()/(1024*1024)
    total_rows+=len(chunk)

print('Total Memory is',total_mem,'mB')
print('Total Number of Rows is ', total_rows)

Total Memory is 66.1424999237 mB
Total Number of Rows is  42538


One of the assumptions of this project is we only have 10mb of memory available to process and analyze this dataset. With a total memory footprint of 66mB we need to ensure each batch of data we process is around 5mb( approx half the 10mb limit, just to be safe)

In [24]:
chunk_iter=pd.read_csv('loans_2007.csv',chunksize=3000)
for chunk in chunk_iter:
    pp.pprint(chunk.memory_usage(deep=True).sum()/(1024*1024))

4.6512298583984375
4.6481781005859375
4.6496686935424805
4.6505937576293945
4.6469202041625977
4.6483755111694336
4.6473674774169922
4.6494159698486328
4.647435188293457
4.6472721099853516
4.6598777770996094
4.6566619873046875
4.6634960174560547
4.8978452682495117
0.88080883026123047


In [29]:
chunk.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
42000,247286,247257.0,6000.0,6000.0,4201.940000,36 months,11.34%,197.40,C,C2,Best Buy,3 years,RENT,32000.0,Not Verified,...,1106.40,0.0,0.00,0.00,Feb-2011,207.82,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,NaN,0.0
42001,246996,244258.0,17250.0,17250.0,12150.005316,36 months,17.66%,620.70,G,G2,CVS PHARMACY,2 years,OWN,62000.0,Not Verified,...,2065.95,0.0,0.00,0.00,Nov-2008,621.01,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,NaN,0.0
42002,246720,246706.0,13000.0,13000.0,7700.000000,36 months,15.13%,451.48,E,E4,General Motors,7 years,RENT,78000.0,Not Verified,...,3253.16,0.0,0.00,0.00,Feb-2011,470.66,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,NaN,0.0
42003,246535,246427.0,12000.0,12000.0,5650.000000,36 months,18.29%,435.58,G,G4,usa medical center,6 years,RENT,62000.0,Not Verified,...,3680.71,0.0,0.00,0.00,Feb-2011,450.84,Jun-2011,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,NaN,0.0
42004,246197,217842.0,4000.0,4000.0,3849.997562,36 months,10.08%,129.22,B,B3,InvestSource Inc,4 years,RENT,48000.0,Not Verified,...,582.00,0.0,61.67,0.85,Mar-2010,129.22,Aug-2010,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,NaN,0.0


There are a few questions we need answered about the data in each column. First what type of data is in each column.

In [25]:
chunk_iter=pd.read_csv('loans_2007.csv',chunksize=3000)
for chunk in chunk_iter:
    pp.pprint(chunk.dtypes.value_counts())

float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     22
dtype: int64
float64    30
object     22
dtype: int64


In [26]:
chunk_iter=pd.read_csv('loans_2007.csv',chunksize=3000)
for chunk in chunk_iter:
    print(chunk.select_dtypes(include=['int64']).columns)

Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index(['id'], dtype='object')
Index([], dtype='object')
Index([], dtype='object')


In general we have 21 object columns and 31 numeric columns. One interesting thing that appears is that the 'ID' changes its dytpe from int64 to object. This should not make a major impact as the ID column is not that important in this project. Next I will check how many unique values are in each object column.

In [28]:
## Create dictionary (key: column, value: list of Series objects representing each chunk's value counts)
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
string_cols_values= {}
for chunk in chunk_iter:
    #select only the object columns
    str_cols = chunk.select_dtypes(include=['object'])
    for col in str_cols.columns:
        #loop through each column and find the value counts
        current_col_vc = str_cols[col].value_counts()
        #save values to dictionary
        if col in string_cols_values:
            string_cols_values[col].append(current_col_vc)
        else:
            string_cols_values[col] = [current_col_vc]


In [16]:
## Create dictionary (key: column, value: list of Series objects representing each chunk's value counts)
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
float_cols_values= {}
float_nan=[]
float_not_nan=[]
for chunk in chunk_iter:
    #select only the object columns
    flt_cols = chunk.select_dtypes(include=['float'])
    for col in flt_cols.columns:
        #save cols with no nan values to list
        if not chunk[col].isnull().any():
            float_not_nan.append(col)
        else:
            float_nan.append(col)
        float_cols_values.update({col:(len(chunk[col]) - chunk[col].count())})
#pp.pprint(sorted(float_cols_values.items(), key=lambda x: x[1])) 
pp.pprint(float_nan)

['pub_rec_bankruptcies',
 'pub_rec_bankruptcies',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'policy_code',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'pub_rec_bankruptcies',
 'tax_liens',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'p

## Optimizing the datatypes for memory efficiency

There are few object columns that we can convert to numeric with a bit of cleaning. The int_rate and revol_util can be converted by removing the percent sign. Can the term column can be converted by removing the months string 

In [42]:
chunk_iter=pd.read_csv('loans_2007.csv',chunksize=3000)
for chunk in chunk_iter:
    chunk['int_rate']= pd.to_numeric(chunk['int_rate'].str.rstrip("%"))
    chunk['revol_util']= pd.to_numeric(chunk['revol_util'].str.rstrip("%")) 
    chunk['term']= pd.to_numeric(chunk['term'].str.lstrip(" ").str.rstrip(" months"))
    pp.pprint(chunk.memory_usage(deep=True).sum()/(1024*1024))                    

4.1684398651123047
4.1656618118286133
4.1671142578125
4.1681051254272461
4.1645698547363281
4.1658296585083008
4.1649141311645508
4.1669502258300781
4.1651468276977539
4.1649971008300781
4.177581787109375
4.1744918823242188
4.1814451217651367
4.4157543182373047
0.79541492462158203


In [43]:
cat_cols = {
    "sub_grade": "category", "home_ownership": "category", 
    "verification_status": "category", "purpose": "category"
}
chunk_iter=pd.read_csv('loans_2007.csv',dtype = cat_cols, parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"],chunksize=3000)
for chunk in chunk_iter:
    
    chunk['int_rate']= pd.to_numeric(chunk['int_rate'].str.rstrip("%"))
    chunk['revol_util']= pd.to_numeric(chunk['revol_util'].str.rstrip("%")) 
    chunk['term']= pd.to_numeric(chunk['term'].str.lstrip(" ").str.rstrip(" months"))
    
    pp.pprint(chunk.memory_usage(deep=True).sum()/(1024*1024))   

2.7836942672729492
2.7802896499633789
2.7825155258178711
2.7827301025390625
2.7799167633056641
2.7808027267456055
2.7801380157470703
2.782109260559082
2.7805881500244141
2.7818393707275391
2.7946376800537109
2.7933759689331055
2.8010787963867188
3.0340814590454102
0.55158805847167969


In [20]:
cat_cols = {
    "sub_grade": "category", "home_ownership": "category", 
    "verification_status": "category", "purpose": "category"
}
chunk_iter=pd.read_csv('loans_2007.csv',dtype = cat_cols, parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"],chunksize=3000)
for chunk in chunk_iter:
    
    chunk['int_rate']= pd.to_numeric(chunk['int_rate'].str.rstrip("%"))
    chunk['revol_util']= pd.to_numeric(chunk['revol_util'].str.rstrip("%")) 
    chunk['term']= pd.to_numeric(chunk['term'].str.lstrip(" ").str.rstrip(" months"))
    flt_cols = chunk.select_dtypes(include=['float'])
    for col in flt_cols.columns:
        if col in float_not_nan:
            chunk[col]= pd.to_numeric(chunk[col], downcast='integer')
        else:
            chunk[col]= pd.to_numeric(chunk[col], downcast='float')
    pp.pprint(chunk.memory_usage(deep=True).sum()/(1024*1024)) 

2.4403715133666992
2.4369668960571289
2.4391927719116211
2.4394073486328125
2.4365940093994141
2.4374799728393555
2.4368152618408203
2.438786506652832
2.4372653961181641
2.4385166168212891
2.4513149261474609
2.4500532150268555
2.4577560424804688
2.6907587051391602
0.49001884460449219


Through optimizing the column datatypes I was able to drop the memory footprint to close to 50% 